# Домашнее задание 1
## Harry Potter and the Action Prediction Challenge from Natural Language


В этом домашнем задании вы будете работать с корпусом Harry Potter and the Action Prediction Challenge. Корпус собран из фанфиков о Гарри Поттере и состоит из двух частей: 1) сырые тексты, 2) фрагменты текстов, описывающих ситуацию, в которой произнесено заклинание.

Корпус описан в статье: https://arxiv.org/pdf/1905.11037.pdf

David Vilares and Carlos Gómez-Rodríguez. Harry Potter and the Action Prediction Challenge from Natural Language. 2019 Annual Conference of the North American Chapter of the Association for Computational Linguistics. To appear.

Код для сбора корпуса находится в репозитории: https://github.com/aghie/hpac . Корпус можно скачать по инструкции из этого репозитория, но для экономии времени авторы задания уже скачали и подготовили данные к работе. 

Ссылки на собранный корпус: 
* Сырые тексты:  https://www.dropbox.com/s/12toaj67fjrguhd/hpac_raw.zip?dl=0
* Токенизированные тексты в нижнем регистре: https://www.dropbox.com/s/1ndto6dce5wg7j2/hpac_lower_tokenized.zip?dl=0
* train-test-dev: https://www.dropbox.com/s/ftinwwjfyydevth/hpac_splits.zip?dl=0

Части 1, 2 задания должны быть выполнены на полных текстах (сырых или предобработанных -- на ваше усмотрение), Часть 3 – на разбиение на тестовое, отладочное и обучающее множества. Тестовое множество должно быть использовано исключительно для тестирования моделей, обучающее и отладочное – для выбора модели и параметров. 

В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется индивидуально.
2. Домашнее задание сдается в системе Anytask, куда можно попасть через инвайт.
3. Домашнее задание оформляется в отчета в ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

## Часть 3. [6.5 баллов] Классификация текстов
Задача классификации формулируется так: данный фрагмент фанфика описывают какую-то ситуацию, которая предшествует произнесению заклинания. Требуется по тексту предсказать, какое именно заклинание будет произнесено. Таким образом, заклинание - это фактически метка класса. Основная мера качества – macro $F_1$.
Обучите несколько классификаторов и сравните их между собой. Оцените качество классификаторов на частых и редких классах. Какие классы чаще всего оказываются перепутаны? Связаны ли ошибки со смыслом заклинаний?

Используйте фрагменты из множества train для обучения, из множества dev для отладки, из множества test – для тестирования и получения итоговых результатов. 

1. [1 балл] Используйте fastText в качестве baseline-классификатора.
2. [2 балла] Используйте сверточные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling.
3. [2 балла] Используйте рекуррентные сети в качестве альтернативного продвинутого классификатора. Поэкспериментируйте с количеством и размерностью слоев и другими гиперпараметрами. 
4. [1.5 балла] Попробуйте расширить обучающее множество за счет аугментации данных. Если вам понадобится словарь синонимов, можно использовать WordNet (ниже вы найдете примеры).

[бонус] Используйте результат max pooling'а как эмбеддинг входного текста. Визуализируйте эмбеддинги 500-1000 предложений из обучающего множества и изучите свойства получившегося пространства.

[бонус] Используйте ваш любимый классификатор и любые (честные) способы повышения качества классификации и получите macro $F_1$ больше 0.5.

## Часть 4. [0.5 балла] Итоги
Напишите краткое резюме проделанной работы. Читали ли вы сами Гарри Поттера или фанфики о нем и помогло ли вам знание предметной области в выполнении домашнего задания?

## Бонусная часть. [2 балла] Skip-Gram Negative Sampling
Самостоятельно реализовать и обучить модель Skip-Gram Negative Sampling. Продемонстрировать качество полученных представлений на конкретный примерах.

# Данные
Сырые тексты 

In [3]:
# !unzip hpac_source

In [4]:
# !ls hpac_source | wc -l

In [5]:
# !unzip hpac_splits

train, test, dev файлы

In [1]:
import pandas as pd

In [2]:
training = pd.read_csv('hpac_splits/hpac_training_128.tsv', sep = '\t', header = None)
test = pd.read_csv('hpac_splits/hpac_test_128.tsv', sep = '\t', header = None)
dev = pd.read_csv('hpac_splits/hpac_dev_128.tsv', sep = '\t', header = None)

In [3]:
df = pd.concat([training, test, dev])

In [13]:
df.iloc[0][1], df.iloc[0][2]

('RIDDIKULUS',
 "were staring at her . she was up next to face the boggart in defense against the dark arts class . she was not scared , but what she was worried about was what had happened with lysander . she looked up at the boggart in front of her which had previously been a humongous spider . its eyes locked on her . before she could think of what frightened her , the spider transformed into lysander . he was dying . there were giggles coming from the male and female hufflepuff students . there was a smirk on lorcan 's face . `` lily help me '' i ca n't fail this class because of a secret love . lily lifted her wand and said , ``")

# Часть 1. [1 балл] Эксплоративный анализ 

[бонус] Постройте тематическую модель по корпусу HPAC.

[бонус] Найдите еще что-то интересное в корпусе (что-то специфичное для фанфиков или фентези-тематики)


## 1. Найдите топ-1000 слов по частоте без учета стоп-слов.

from collections import Counter 

In [141]:
from collections import Counter
import nltk
import ssl

# фиксим траблы с сертификатами
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruattar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
all_text = ' '.join(df[2].to_list())

In [9]:
filter_func = lambda x: x not in stopwords and x.isalpha()

In [10]:
word_cnt = Counter(filter(filter_func, all_text.split(' ')))

In [11]:
commom_1000 = word_cnt.most_common(1000)

In [12]:
commom_1000[:5]

[('harry', 59043),
 ('wand', 42489),
 ('said', 29459),
 ('could', 28027),
 ('would', 25913)]

## 2 Найдите топ-10 по частоте: имен, пар имя + фамилия

Распарсим имена

https://gist.github.com/jennynz/7eaf7ea4eeb3d686b19e997e721bda0c

In [114]:
# list(filter(lambda x: x[0] in ('mr', 'mrs', 'ms', 'miss'), word_cnt.items()))

In [13]:
with open('Harry Potter names', 'r') as f:
    namelines = f.read().split('\n')

In [14]:
def parse_namelist(s):
    resdict = {
        'name': None,
        'surname': None,
        'middlename': None,
        'title': None,
    }

    if len(s) == 0:
        return resdict
    
    l = s.lower().split(' ')

    if len(l) == 1:
        resdict['name'] = resdict['surname'] = l[0]
        
    elif len(l) > 1:
        
        if l[0] in ('mr', 'miss', 'mrs', 'ms', 'lord'):
            resdict['title'] = l[0]
            l = l[1:]
        
        resdict['name'] = l[0]    
        resdict['surname'] = l[-1]
        
    if len(l) > 2:
        resdict['middlename'] = ' '.join(l[1:-1])
        
    return resdict

In [53]:
parsed_names = pd.DataFrame(map(parse_namelist, namelines))

new_row = {
    'name': 'ginny',
    'surname': "weasley",
    'middlename': None,
    'title': None,
}

parsed_names = parsed_names._append(new_row, ignore_index=True)

In [28]:
names = parsed_names.name.unique()

In [31]:
popular_names = sorted(
    list(filter(lambda x: x[0] in names, word_cnt.items())),
    key=lambda x: x[1],
    reverse=True
)

Топ популярных имен:

In [33]:
popular_names[:5]

[('harry', 59043),
 ('hermione', 24062),
 ('draco', 21266),
 ('voldemort', 16091),
 ('sirius', 9929)]

Ищем имя + фамилия

In [51]:
import re
text = df[df[2].str.contains('harry potter')].iloc[19, 2]
re.findall('harry potter', text)

['harry potter']

In [106]:
import re  
from collections import defaultdict
from typing import List
  
def count_mentions(names: List[str], text: str, mentions: defaultdict):
    """  
    Считает количество упоминаний имен и фамилий в тексте и добавляет их в mentions  
  
    Args:  
        names (list): Список строк вида "имя фамилия".  
        text (str): Текст, в котором нужно найти упоминания.  
        mentions (defaultdict): словарь счетчик
    Returns:  None  
    """  
    for name in tqdm_notebook(names):  
        pattern = re.compile(r'\b' + re.escape(name) + r'\b', re.IGNORECASE)  
        mentions[name] += len(pattern.findall(text))   

Так, упоминание имя + фамилия это подможество упоминаний имени, поэтому чтобы не искать по всем именам и фамилиям, можем список name_surname_lst фильтрануть на топ имен

In [121]:
top_30_names = [i[0] for i in popular_names[:30]]

In [130]:
name_surname_df = parsed_names.query("name.isin(@top_30_names)")
name_surname_lst = sorted(
    list(
        set(
            name_surname_df['name'] + ' ' + name_surname_df['surname'].str.replace(')', '')
        )
    )
)

In [133]:
len(name_surname_lst)

40

In [105]:
# from tqdm import tqdm_notebook

# for _, row in tqdm_notebook(df.iterrows()):
#     count_mentions(name_surname_lst, row[2], MENTIONS)

In [136]:
MENTIONS = defaultdict(int)

count_mentions(name_surname_lst, all_text, MENTIONS)

/var/folders/yh/4jky_nln065c8nrzc6phgjjdt0k6bs/T/ipykernel_69006/1668062734.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(names):


  0%|          | 0/40 [00:00<?, ?it/s]

In [140]:
sorted(list(MENTIONS.items()), key=lambda x: x[1], reverse=True)[:10]

[('harry potter', 3178),
 ('draco malfoy', 1058),
 ('james potter', 762),
 ('severus snape', 713),
 ('hermione granger', 634),
 ('lucius malfoy', 629),
 ('sirius black', 499),
 ('albus dumbledore', 378),
 ('remus lupin', 290),
 ('lily evans', 266)]

<!-- ## [bonus] Постройте тематическую модель по корпусу HPAC. -->

# Часть 2. [2 балла] Модели представления слов 
Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.
1. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели. 
2. Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP (https://umap-learn.readthedocs.io).

In [144]:
# !pip install gensim

# Допы

### Как использовать WordNet из nltk?

In [ ]:
# скачиваем WordNet
import nltk
nltk.download('wordnet')

In [ ]:
# слово -> множество синсетов (синонимов разных смыслов исходного слова)
from nltk.corpus import wordnet as wn
wn.synsets('magic')

In [ ]:
# посмотрим, что внутри одного синсета
wn.synsets('magic')[1].lemmas()[0]

In [ ]:
# возьмем лемму одного из слов из синсета
wn.synsets('magic')[1].lemmas()[-1].name()